## 概述：该文档包含的代码实现了...：

### 1 处理A：2015年每日数据插入空白行，形成一天360行的数据的结构
### 2 处理B：将补齐后的文件索取数据填入空白表

## 所使用的技巧：
### 1 empty_series[i] = sub_set.loc[i]['2362A'] if i in sub_set.index else np.nan 
### 2 loc[***] = 某个list 通过列表的形式填入数据内容。（当loc索引的位置不存在可以新建，而iloc不能这样操作）

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline
import os

#### 1.1 读取2015年文件名称列表

In [2]:
name_list = os.listdir('/mnt/e/_03_data_air/sties/站点_20150101-20151231')
# name_list # 用于查看

#### 1.2 补齐操作

In [3]:
hour_list = np.arange(0, 24)  # 用于后续小时判断
for name in os.listdir('/mnt/e/_03_data_air/sties/站点_20150101-20151231'): # 可以用上面的name_list替换
    # 从os.listdir是文件名称列表，通过名称变换实现日期变换
    path = os.path.join('/mnt/e/_03_data_air/sties/站点_20150101-20151231', name)  # 拼接文档路径和文件名称形成文件路径
    
    df = pd.read_csv(path, encoding='UTF-8')  # 打开当天文件，且命名为df
    
    need_processing = False  # 是否需要处理的标志，设置为否！！！！！！！！！！！！！！！

    for i in hour_list:
        count_num = df[df['hour'] == i]['date'].count()
        # “df[df['hour'] == i]”中间的 == 会产生一个bool索引。导入bool索引之后对其date列有多少数据进行统计。
        # date也可以换成别的，但是date一定有数据（因为只有当date一列其他所有监测值为空date则跳过）

        if count_num < 15:
            need_processing = True  # 设置标志为 True，表示需要处理！！！！！！！！！！
            insert_index = i * 15  # 插入的索引位置
            
            empty_row = pd.Series(index=df.columns)
            # 空白行模版构建，series和df拼接是需要转换，见后面。
            # 此外df.columns获取列名称，df在前面就已经打开了，所以可以获取，列名称实际上是一些站点的代号，这些操作用于df转换。

            date_value = df['date'].iloc[0]  # 这 是在for循环中某个i小时的操作，因此，该段代码为获取当前小时日期数据。
            empty_row['date'] = date_value  # 赋值给空series，中日期数据。
            empty_row['hour'] = i  # 赋值小时数据

            df = pd.concat([df.iloc[:insert_index], pd.DataFrame([empty_row] * (15 - count_num), columns=df.columns),df.iloc[insert_index:]]).reset_index(drop=True)
            # pd.DataFrame——将series转化为df才能拼接
            # 拼接三个部分insert_index之前（不包括该行），新增行和insert_index之后行
            # reset_index(drop=True) 是用于重置DataFrame的索引，并丢弃原来的索引，如果不丢弃，原来索引会变成一列新数据。
            # 插入的行数为空白行×倍数“15-count_num”

    # 如果需要处理，则将修改后的 DataFrame 写入新文件
    if need_processing:
        df.to_csv(f'/mnt/e/_03_data_air/2015修改/{name}', index=False)

## 2

#### 2.1 新建空白表

In [4]:
# 定义新建表的列名称、站点名称
columns_name = ['date','hour','AQI', 'PM2.5', 'PM10', 'SO2', 'NO2', 'O3', 'O3_8h', 'CO']
sties_name = ['2362A','2363A','2364A','2365A','2366A'] # 循环站点列表

# 定义空白表单：用于全年数据汇总空白表
new_df_allyear = pd.DataFrame(columns = columns_name)

# 定义（获取）date路径文件名单列表
site_file_name_list = os.listdir('/mnt/e/_03_data_air/sties/站点_20150101-20151231')

# 定义索引：用于截取正常情况下的数据
index_list = [0] + [2 * i - 1 for i in np.arange(1, 8)] ## list可以直接相加

#### 2.2  关于每小时数据在原始文件的索引
##### 首先，我们知道第一行是0号索引（除表头），因此原文件361行实际上索引区间为0~359
##### 此外每小时一共有15组数据，因此0~14代表第个小时的全部数据，但是索引区间是一个半开半闭区间[A,B)。
##### 因此第一个小时[0, 15)，最后一个小时为[23*15, 24*15)

In [5]:
for sites in sties_name: # 循环处理每个站点（site）
    
    ############## 空白文档：年汇总 ####################
    df_all_years_date = pd.DataFrame(columns=columns_name)

    for file_name in site_file_name_list: # 循环处理每个文件（file）
        
        #打开文件，手动填写路径
        path = os.path.join('/mnt/e/_03_data_air/sties/站点_20150101-20151231', file_name) 
        df = pd.read_csv(path)
        
        # 获取日期数据，可以不使用iloc
        date_value = df['date'].iloc[0]
        
        ################## 空白文档：日汇总 ############
        df_day_date = pd.DataFrame(columns=columns_name)

        for hour in np.arange(0, 24, 1): # 循环每个小时
            
            # num：每小时有效数据量
            # 具体技巧：df切片hour列→得到series（只包含小时数据）， → 利用==获取bool索引，导入df[]中（或df.loc[]中）切片→特定小时数据df
            # df再切片成ytpe列→得到series（只包含数据类型）→进行count（）计算
            num = df[df['hour'] == hour]['type'].count()
            
            # 获取第‘hour’小时数据索引区间。
            start_num = hour * 15
            end_num = (hour + 1) * 15

            if num < 15:
                sub_set = df.iloc[start_num:end_num][['type',sites]].set_index('type') # type存在空，但是可以被设置为索引
                
                # 后面将使用for循环填写empty_series（空白列）
                empty_series = pd.Series(index=columns_name)

                for i in columns_name: # i表示列名称 
                                        
                    # 将sub_set中相关数据填入到empty_series中
                    # if后的技巧NaN作为索引是，相当于没有索引。（索引NaN表示没有索引）
                    empty_series[i] = sub_set.loc[i][sites] if i in sub_set.index else np.nan
                    # empty_series[i] = sub_set.loc[i, sites] if i in sub_set.index else np.nan

                # 补齐空series中date和hour
                empty_series['date'] = date_value
                empty_series['hour'] = hour

                # 转置为df结构，用于后续concat, empty_series事实上这里已经有数据了
                add_df = pd.DataFrame(empty_series).T
                df_day_date = pd.concat([df_day_date, add_df], axis=0, ignore_index=True)

            elif num == 15:
                sub_set = df.iloc[start_num:end_num].iloc[index_list][sites] # →series
                
                # 在df的新的一行填写数据，hour实际上表示第几行。
                # loc[***] 实际上可以通过列表的形式填写数据内容。
                df_day_date.loc[hour] = [date_value] + [hour] + sub_set.values.tolist()

        df_all_years_date = pd.concat([df_all_years_date, df_day_date], axis=0, ignore_index=True) #axis = 0表示行方向

    # 补充缺失日期，date有一个小数点是为了格式一致
    missing_date = '20150101.0'
    missing_hours = list(range(24))
    missing_data = {'date': [missing_date] * 24, 'hour': missing_hours}
    missing_df = pd.DataFrame(missing_data)
    df_all_years_date = pd.concat([missing_df, df_all_years_date], ignore_index=True)

    # 保存到 Excel 文件（同一个文件，不同 sheet）
    # 在pd.ExcelWriter方法中，如果原本不存在该文件会自动生成
    # mode='a'表示追加数据模式，engine表示写入引擎
    file_path = f'/mnt/e/_03_data_air/all_2015.xlsx'############## 生成文件路径（自定义） ############################################
    try:
        writer = pd.ExcelWriter(file_path, engine='openpyxl', mode='a')
        
        # index = False表示不写入excel中
        df_all_years_date.to_excel(writer, index=False, sheet_name=f'{sites}')
        writer.close()
    except Exception as e:
        print(f"发生错误: {e}")
        # 处理文件不存在的情况
        df_all_years_date.to_excel(file_path, index=False, sheet_name=f'{sites}')


/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_day_date = pd.concat([df_day_date, add_df], axis=0, ignore_index=True)
/tmp/ipykernel_786/3621910682.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all_years_date = pd.concat([df_all_years_date, df_day_date], axis=0, ignore_index=True) #axis = 0表示行方向
/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future versio

/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_day_date = pd.concat([df_day_date, add_df], axis=0, ignore_index=True)
/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_day_date = pd.concat([df_day_date, add_df], axis=0, ignore_index=True)
/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude e

发生错误: [Errno 2] No such file or directory: '/mnt/e/_03_data_air/all_year.xlsx'


/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_day_date = pd.concat([df_day_date, add_df], axis=0, ignore_index=True)
/tmp/ipykernel_786/3621910682.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all_years_date = pd.concat([df_all_years_date, df_day_date], axis=0, ignore_index=True) #axis = 0表示行方向
/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future versio

/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_day_date = pd.concat([df_day_date, add_df], axis=0, ignore_index=True)
/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_day_date = pd.concat([df_day_date, add_df], axis=0, ignore_index=True)
/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude e

/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_day_date = pd.concat([df_day_date, add_df], axis=0, ignore_index=True)
/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_day_date = pd.concat([df_day_date, add_df], axis=0, ignore_index=True)
/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude e

/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_day_date = pd.concat([df_day_date, add_df], axis=0, ignore_index=True)
/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_day_date = pd.concat([df_day_date, add_df], axis=0, ignore_index=True)
/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude e

/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_day_date = pd.concat([df_day_date, add_df], axis=0, ignore_index=True)
/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_day_date = pd.concat([df_day_date, add_df], axis=0, ignore_index=True)
/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude e

/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_day_date = pd.concat([df_day_date, add_df], axis=0, ignore_index=True)
/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_day_date = pd.concat([df_day_date, add_df], axis=0, ignore_index=True)
/tmp/ipykernel_786/3621910682.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude e

### 调试代码

NumPy默认浮点数数据类型: <class 'numpy.float64'>


Pandas默认浮点数数据类型: float64
